# load library

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support, classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



# load data

In [14]:
# 1. 데이터프레임 로드
cluster_df = pd.read_csv('preprocessed data/cluster_df.csv', index_col=0)
cluster_df = cluster_df[cluster_df['dbscan_cluster']!=-1]
cluster_df

,RESPOND_ID,EXAMIN_BEGIN_DE,SEX,AGE,INCOME,SEOUL,GYEONGGI,BUSAN,CHUNGNAM,CHUNGBUK,...,Occupation_경영/관리/전문직,Occupation_기능/숙련/일반작업직,Occupation_기타/무직,Occupation_대학/대학원생,Occupation_사무/기술직,Occupation_자영업,Occupation_전업주부,Occupation_판매/서비스직,dbscan_cluster,kmeans_cluster
0,53364346,20240101,0,50,1,3,4,3,3,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
1,53363665,20240101,0,50,0,3,3,4,3,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,73,1
2,53322894,20240101,0,50,1,3,4,4,3,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,44,1
6,53328071,20240101,1,40,1,3,3,3,3,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,34,1
9,53326024,20240101,0,20,0,3,1,3,1,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,53350730,20241230,0,50,0,1,4,3,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35,1
1566,53364632,20241230,1,50,1,3,4,4,1,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11,1
1570,53367863,20241230,0,40,1,1,2,4,2,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2,1
1571,53369696,20241230,1,50,1,3,4,1,1,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1


# DecisionTreeClassifier

### DecisionTreeClassifier prediction for dbscan cluster

In [15]:
# 1) feature / target 분리
X = cluster_df.iloc[:, 2:-2].to_numpy()
y = cluster_df.iloc[:,-2:-1].values.reshape(-1)
# 2) train + temp (validation + test) 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y)

# 3) validation / test 분리 (temp 40% -> 20% val + 20% test)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

print(f"train set size: {X_train.shape[0]} samples")
print(f"validation set size: {X_val.shape[0]} samples")
print(f"test set size: {X_test.shape[0]} samples")

# 4) 모델 학습 및 검증 예시
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# 검증 데이터 예측 및 평가
y_val_pred = model.predict(X_val)
print("Validation set performance:")
print(classification_report(y_val, y_val_pred, zero_division=0))

# 테스트 데이터 평가
y_test_pred = model.predict(X_test)
print("Test set performance:")
print(classification_report(y_test, y_test_pred, zero_division=0))


train set size: 603 samples
validation set size: 201 samples
test set size: 202 samples
Validation set performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      0.67      0.80         9
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         1
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         5
           8       0.75      0.75      0.75         4
           9       0.80      1.00      0.89         4
          10       1.00      1.00      1.00         4
          11       1.00      1.00      1.00         6
          12       1.00      1.00      1.00         7
          13       0.40      0.67      0.50         3
          14       0.67      1.00      0.80         2
          15       

### DecisionTreeClassifier prediction for kmeans cluster(다양한 페르소나의 군집을 예측하는 것이 목표이기에 의미를 두지 않음)

In [16]:
# 1) feature / target 분리
X = cluster_df.iloc[:, 2:-2].to_numpy()
y = cluster_df.iloc[:,-1:].values.reshape(-1)
# 2) train + temp (validation + test) 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y)

# 3) validation / test 분리 (temp 40% -> 20% val + 20% test)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

print(f"train set size: {X_train.shape[0]} samples")
print(f"validation set size: {X_val.shape[0]} samples")
print(f"test set size: {X_test.shape[0]} samples")

# 4) 모델 학습 및 검증 예시
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# 검증 데이터 예측 및 평가
y_val_pred = model.predict(X_val)
print("Validation set performance:")
print(classification_report(y_val, y_val_pred, zero_division=0))

# 테스트 데이터 평가
y_test_pred = model.predict(X_test)
print("Test set performance:")
print(classification_report(y_test, y_test_pred, zero_division=0))


train set size: 603 samples
validation set size: 201 samples
test set size: 202 samples
Validation set performance:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        52
           1       0.99      1.00      1.00       149

    accuracy                           1.00       201
   macro avg       1.00      0.99      0.99       201
weighted avg       1.00      1.00      1.00       201

Test set performance:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        63
           1       0.99      1.00      1.00       139

    accuracy                           1.00       202
   macro avg       1.00      0.99      0.99       202
weighted avg       1.00      1.00      1.00       202



### DecisionTreeClassifier prediction for kmeans cluster & dbscan cluster

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support, classification_report

# 1) feature / target 분리 (타겟 여러 컬럼)
X = cluster_df.iloc[:, 2:]
y = cluster_df.iloc[:, -2:]  # 다중 라벨 (2개)

# 2) train + temp 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42)

# 3) val / test 분리
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

# 4) 모델 학습
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# 5) 검증 데이터 예측
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# 6) 평가 함수 정의
from sklearn.metrics import precision_recall_fscore_support

def print_scores(y_true, y_pred, set_name=""):
    # 각 클래스별 score와 support
    precisions, recalls, f1s, supports = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average=None, zero_division=0
    )
    for i, (p, r, f, s) in enumerate(zip(precisions, recalls, f1s, supports)):
        print(f"[{set_name} Class {i}] Precision: {p:.4f}, Recall: {r:.4f}, F1: {f:.4f}, Support: {s}")

    # macro, weighted score
    p_macro, r_macro, f_macro, _ = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average='macro', zero_division=0
    )
    p_weighted, r_weighted, f_weighted, _ = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average='weighted', zero_division=0
    )
    print(f"[{set_name} Macro]    Precision: {p_macro:.4f}, Recall: {r_macro:.4f}, F1: {f_macro:.4f}")
    print(f"[{set_name} Weighted] Precision: {p_weighted:.4f}, Recall: {r_weighted:.4f}, F1: {f_weighted:.4f}")

# 7) 검증/테스트 세트 평가지표 출력
print_scores(y_val.values, y_val_pred, set_name="Validation")
print_scores(y_test.values, y_test_pred, set_name="Test")


[Validation Class 0] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 69
[Validation Class 1] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 144
[Validation Class 2] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 3
[Validation Class 3] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 6
[Validation Class 4] Precision: 0.5000, Recall: 1.0000, F1: 0.6667, Support: 1
[Validation Class 5] Precision: 1.0000, Recall: 0.8333, F1: 0.9091, Support: 6
[Validation Class 6] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 6
[Validation Class 7] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 3
[Validation Class 8] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 2
[Validation Class 9] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 10
[Validation Class 10] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 5
[Validation Class 11] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 5
[Validation Class 12] Precision: 1.0000, Recal

# Random Forest

### Random Forest for dbscan cluster

In [18]:
# 1) feature / target 분리
X = cluster_df.iloc[:, 2:-2].to_numpy()
y = cluster_df.iloc[:, -2:-1].values.reshape(-1)

# 2) train + temp (validation + test) 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# 3) validation / test 분리 (temp 40% -> 20% val + 20% test)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(f"train set size: {X_train.shape[0]} samples")
print(f"validation set size: {X_val.shape[0]} samples")
print(f"test set size: {X_test.shape[0]} samples")

# 4) 모델 학습 및 검증 예시 (Random Forest)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 검증 데이터 예측 및 평가
y_val_pred = model.predict(X_val)
print("Validation set performance:")
print(classification_report(y_val, y_val_pred, zero_division=0))

# 테스트 데이터 평가
y_test_pred = model.predict(X_test)
print("Test set performance:")
print(classification_report(y_test, y_test_pred, zero_division=0))


train set size: 603 samples
validation set size: 201 samples
test set size: 202 samples
Validation set performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      0.89      0.94         9
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         3
           4       0.86      1.00      0.92         6
           5       1.00      1.00      1.00         1
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         5
           8       1.00      0.75      0.86         4
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00         4
          11       1.00      1.00      1.00         6
          12       0.88      1.00      0.93         7
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00         2
          15       

### RandomForest for kmeans cluster & dbscan cluster

In [ ]:
# 1) feature / target 분리 (타겟 여러 컬럼)
X = cluster_df.iloc[:, 2:]
y = cluster_df.iloc[:, -2:]  # 다중 라벨 (2개)

# 2) train + temp 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)

# 3) val / test 분리
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# 4) 모델 학습 (Random Forest로 변경)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 5) 검증 데이터 예측
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# 6) 평가 함수 정의
def print_scores(y_true, y_pred, set_name=""):
    # 각 클래스별 score와 support
    precisions, recalls, f1s, supports = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average=None, zero_division=0
    )
    for i, (p, r, f, s) in enumerate(zip(precisions, recalls, f1s, supports)):
        print(f"[{set_name} Class {i}] Precision: {p:.4f}, Recall: {r:.4f}, F1: {f:.4f}, Support: {s}")

    # macro, weighted score
    p_macro, r_macro, f_macro, _ = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average='macro', zero_division=0
    )
    p_weighted, r_weighted, f_weighted, _ = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average='weighted', zero_division=0
    )
    print(f"[{set_name} Macro]    Precision: {p_macro:.4f}, Recall: {r_macro:.4f}, F1: {f_macro:.4f}")
    print(f"[{set_name} Weighted] Precision: {p_weighted:.4f}, Recall: {r_weighted:.4f}, F1: {f_weighted:.4f}")

# 7) 검증/테스트 세트 평가지표 출력
print_scores(y_val.values, y_val_pred, set_name="Validation")
print_scores(y_test.values, y_test_pred, set_name="Test")


[Validation Class 0] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 69
[Validation Class 1] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 144
[Validation Class 2] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 3
[Validation Class 3] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 6
[Validation Class 4] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 1
[Validation Class 5] Precision: 1.0000, Recall: 0.8333, F1: 0.9091, Support: 6
[Validation Class 6] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 6
[Validation Class 7] Precision: 0.7500, Recall: 1.0000, F1: 0.8571, Support: 3
[Validation Class 8] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 2
[Validation Class 9] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 10
[Validation Class 10] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 5
[Validation Class 11] Precision: 1.0000, Recall: 1.0000, F1: 1.0000, Support: 5
[Validation Class 12] Precision: 1.0000, Recal

# Neural Networks, MLP

### MLP for dbscan cluster

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm

# 데이터 분리 (기존과 동일)
X = cluster_df.iloc[:, 2:-2].to_numpy()
y = cluster_df.iloc[:, -2:-1].values.reshape(-1)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# 텐서 변환
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# DataLoader 준비
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# MLP 모델 정의
n_inputs = X_train.shape[1]
n_classes = len(np.unique(y))

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
    def forward(self, x):
        return self.layers(x)

model = MLP(n_inputs, n_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# 학습 루프
n_epochs = 1000
for epoch in tqdm(range(n_epochs)):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

# 검증 및 테스트 평가 함수
def evaluate(model, loader):
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for xb, yb in loader:
            outputs = model(xb)
            pred = outputs.argmax(dim=1).cpu().numpy()
            true = yb.cpu().numpy()
            preds.append(pred)
            trues.append(true)
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    return trues, preds

# 성능 출력
y_val_true, y_val_pred = evaluate(model, val_loader)
print("Validation set performance:")
print(classification_report(y_val_true, y_val_pred, zero_division=0))

y_test_true, y_test_pred = evaluate(model, test_loader)
print("Test set performance:")
print(classification_report(y_test_true, y_test_pred, zero_division=0))


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [01:27<00:00, 11.41it/s]

Validation set performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      0.89      0.94         9
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         3
           4       0.86      1.00      0.92         6
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         2
           7       1.00      0.80      0.89         5
           8       0.75      0.75      0.75         4
           9       0.80      1.00      0.89         4
          10       1.00      1.00      1.00         4
          11       1.00      1.00      1.00         6
          12       1.00      1.00      1.00         7
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         1
          16       0.83      1.00      0.91         5

### mlp for kmeans cluster & dbscan cluster

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# 1) feature / target 분리 (타겟 여러 컬럼)
X = cluster_df.iloc[:, 2:].to_numpy(dtype=np.float32)
y = cluster_df.iloc[:, -2:].to_numpy(dtype=np.float32)  # 다중 라벨 (2개 이상 assumed)

# 2) train + temp 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)

# 3) val / test 분리
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# 텐서 변환
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# DataLoader 준비
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=32)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32)

# 4) 모델 정의 (Multi-label에 맞는 출력층)
n_inputs = X_train.shape[1]
n_targets = y_train.shape[1]

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
    def forward(self, x):
        return self.layers(x)  # BCEWithLogitsLoss는 sigmoid 미적용 출력 기대

model = MLP(n_inputs, n_targets).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()  # 멀티라벨 이진 분류용

# 5) 학습 루프
n_epochs = 10000
for epoch in tqdm(range(n_epochs)):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

# 6) 평가 함수
def get_preds_targets(model, loader):
    model.eval()
    preds_list = []
    trues_list = []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            outputs = model(xb)
            probs = torch.sigmoid(outputs)     # 확률로 변환
            preds = (probs > 0.5).cpu().numpy().astype(int)
            preds_list.append(preds)
            trues_list.append(yb.cpu().numpy())
    preds = np.concatenate(preds_list, axis=0)
    trues = np.concatenate(trues_list, axis=0)
    return trues, preds

# 7) 성능 평가 (기존 print_scores 함수 사용)
def print_scores(y_true, y_pred, set_name=""):
    # flatten해서 평가 (멀티라벨 평균)
    precisions, recalls, f1s, supports = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average=None, zero_division=0
    )
    for i, (p, r, f, s) in enumerate(zip(precisions, recalls, f1s, supports)):
        print(f"[{set_name} Class {i}] Precision: {p:.4f}, Recall: {r:.4f}, F1: {f:.4f}, Support: {s}")

    p_macro, r_macro, f_macro, _ = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average='macro', zero_division=0
    )
    p_weighted, r_weighted, f_weighted, _ = precision_recall_fscore_support(
        y_true.ravel(), y_pred.ravel(), average='weighted', zero_division=0
    )
    print(f"[{set_name} Macro]    Precision: {p_macro:.4f}, Recall: {r_macro:.4f}, F1: {f_macro:.4f}")
    print(f"[{set_name} Weighted] Precision: {p_weighted:.4f}, Recall: {r_weighted:.4f}, F1: {f_weighted:.4f}")

# 8) 검증/테스트 세트 지표 출력
y_val_true, y_val_pred = get_preds_targets(model, val_loader)
print_scores(y_val_true, y_val_pred, set_name="Validation")
y_test_true, y_test_pred = get_preds_targets(model, test_loader)
print_scores(y_test_true, y_test_pred, set_name="Test")


100%|██████████| 10000/10000 [13:39<00:00, 12.20it/s]


[Validation Class 0] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 69
[Validation Class 1] Precision: 0.3582, Recall: 1.0000, F1: 0.5275, Support: 144
[Validation Class 2] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 3
[Validation Class 3] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 6
[Validation Class 4] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 1
[Validation Class 5] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 6
[Validation Class 6] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 6
[Validation Class 7] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 3
[Validation Class 8] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 2
[Validation Class 9] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 10
[Validation Class 10] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 5
[Validation Class 11] Precision: 0.0000, Recall: 0.0000, F1: 0.0000, Support: 5
[Validation Class 12] Precision: 0.0000, Recal

In [29]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# 1) 데이터 준비
X = cluster_df.iloc[:, 2:-2].to_numpy()
y = cluster_df.iloc[:, -2].to_numpy()   # y가 1개 레이블일 때

# 2) 데이터 분리
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 3) 나이브 베이즈 모델 적합
model = GaussianNB()
model.fit(X_train, y_train)

# 4) 예측 및 평가
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

print("Validation set performance:")
print(classification_report(y_val, y_val_pred, zero_division=0))
print("Test set performance:")
print(classification_report(y_test, y_test_pred, zero_division=0))


Validation set performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      0.89      0.94         9
           2       1.00      1.00      1.00         5
           3       0.75      1.00      0.86         3
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         2
           7       0.83      1.00      0.91         5
           8       0.50      0.50      0.50         4
           9       0.67      1.00      0.80         4
          10       1.00      1.00      1.00         4
          11       1.00      1.00      1.00         6
          12       1.00      0.86      0.92         7
          13       0.75      1.00      0.86         3
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         1
          16       1.00      1.00      1.00         5